In [27]:
import pandas as pd
df = pd.read_csv ('./session/log3.csv') # import list of all WiFi devices from CSV file

In [28]:
import re
file = open('../mac-vendors.txt', 'r', encoding="utf8")
mac_dict = { mac[0]: mac[1] for mac in [re.split(r'\t+', line.strip()) for line in file] } # split file by tabs into dictionary
def getDeviceType(bssid): return mac_dict.get(bssid[:8]) # trim first 3 octets from MAC

In [38]:
creep_threshold = .5  # creep detection threshold distance in miles
creep_list = []

from haversine import haversine
import itertools

for mac in df['MAC'].unique():
    ssid_list = df.loc[df['MAC'] == mac]
    gps_coords = df.loc[df['MAC'] == mac][['Latitude', 'Longitude']].values.tolist() # list of all GPS coords per device  
    distance = [haversine(a,b, unit='mi') for a, b in itertools.combinations(gps_coords, 2)] # distances between every single point
    try:
        if(max(distance) > creep_threshold):
            creep_list.append([mac,ssid_list['SSID'].values[0],ssid_list['Encryption'].values[0],ssid_list['Channel'].values[0],ssid_list['RSSI'].mean(),ssid_list['AltitudeMeters'].mean(),max(distance),[ssid_list['Latitude'].values[0],ssid_list['Longitude'].values[0]],ssid_list['Seen'].values[0]])
    except ValueError: pass
creeps = pd.DataFrame(creep_list, columns=['MAC','SSID',"Encryption","Channel","Avg RSSI","Avg Altitude (m)","Max Dist (mi)","First Seen At","First Time"])
creeps.style.set_table_attributes("style='display:inline'").set_caption('Creeps Spotted with Threshold '+str(creep_threshold)+" Miles")

,MAC,SSID,Encryption,Channel,Avg RSSI,Avg Altitude (m),Max Dist (mi),First Seen At,First Time
0,8E:AA:B5:1C:31:3E,Bingus Headquarters,WPA2,1,-41.583333,343.907895,2.856214,"[33.684306, -117.619042]",2021-7-19 14:34:6
1,4E:66:41:9A:81:4B,Hot Bitch Network,WPA2,6,-52.449275,366.271014,2.183428,"[33.684306, -117.619042]",2021-7-19 14:34:6
2,69:AA:B5:1C:31:3E,nan,WPA2,1,-69.000000,342.700000,2.986495,"[33.684306, -117.619042]",2021-7-19 14:34:6


In [39]:
import folium
from folium import FeatureGroup, LayerControl, Map, Marker
myMap = folium.Map(df[['Latitude', 'Longitude']].mean().values.tolist())
folium.PolyLine(df[['Latitude', 'Longitude']].values.tolist(),line_opacity = 0.5, weight = 4).add_to(myMap)
myMap.fit_bounds([df[['Latitude', 'Longitude']].min().values.tolist(), df[['Latitude', 'Longitude']].max().values.tolist()])

df_add = df.loc[df['SSID'].isin(creeps['SSID'].values.tolist())]
colors = [ 'red', 'blue', 'green', 'orange', 'purple', 'beige', 'gray', 'pink', 'black', 'lightgreen', 'darkblue', 'lightblue', 'lightred', 'darkpurple', 'darkred', 'cadetblue', 'lightgray', 'darkgreen' ]

for i,(grp_name, df_grp) in enumerate(df_add.groupby('SSID')):
    feature_group = folium.FeatureGroup(grp_name)
    for row in df_grp.itertuples():
        folium.Marker(location=[row.Latitude, row.Longitude],  popup="bingus", icon=folium.Icon(color=colors[i % len(colors)],icon='user-secret',prefix='fa')).add_to(feature_group)
    feature_group.add_to(myMap)

folium.LayerControl().add_to(myMap)
display(myMap)